#### Índice
1. Sobre o conjunto de dados
2. Pré-processamento de texto
3. Usar TFIDF para melhorar o vetor de contagem
4. Conclusão

### 1. Sobre o dataset 

CEPs

Há um total de xxx CEPs no conjunto de dados..


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

#import nltk
#nltk.download('rslp')

import unicodedata
from multiprocessing import  Pool
import re

pd.set_option('display.max_colwidth', None)

In [2]:
arquivo = 'ect_amostra_minas_gerais.csv'
sep = ';'
# ler o dataset
df = pd.read_csv(arquivo, sep=sep)
#df.select_dtypes(exclude=[np.number]).columns
#df.shape
df.head()

,log_nu,ufe_sg,loc_nu,bai_nu_ini,bai_nu_fim,log_no,log_complemento,cep,int_cep,tlo_tx,log_sta_tlo,log_no_abrev,bai_no,bai_no_abrev,mun_nu
0,1008337,MG,4047,57341,0,Ouro Fino,NaN,38042274,38042274,Rua,S,R Ouro Fino,Damha Residencial Uberaba I,D Res Uberaba I,3170107
1,1008338,MG,4047,57341,0,Mariana,NaN,38042276,38042276,Rua,S,R Mariana,Damha Residencial Uberaba I,D Res Uberaba I,3170107
2,1008339,MG,4047,57341,0,Lagoa Formosa,NaN,38042278,38042278,Rua,S,R Lga Formosa,Damha Residencial Uberaba I,D Res Uberaba I,3170107
3,1008340,MG,4047,57341,0,A,NaN,38042280,38042280,Rua,S,R A,Damha Residencial Uberaba I,D Res Uberaba I,3170107
4,1008341,MG,4047,57341,0,Um,NaN,38042282,38042282,Avenida,S,Av Um,Damha Residencial Uberaba I,D Res Uberaba I,3170107


In [3]:
df.shape

(111880, 15)

In [4]:
# https://pt.stackoverflow.com/questions/375592/como-cortar-texto-de-string-em-python-delimitando-a-substring-inicial-para-a-po
def split_text(obj, substring = None, start = 0, qtd = None):
    qtd = len(obj) if qtd is None else qtd

    if substring:
        inicio = obj.find(substring)
        return obj[inicio:inicio+qtd]
    elif not substring:
        return obj[start:start+qtd]

In [5]:
# Coluna label_cep conté os 3 primeiros dígitos do CEP. Junto com ufe_sg label_cep forma o par
# para a estratificação proporcional dos dados.
df['label_cep'] = (df['cep']/100000).astype(int)

features_all = ['log_nu', 'ufe_sg', 'loc_nu', 'bai_nu_ini', 'bai_nu_fim', 'log_no', 
            'log_complemento', 'cep', 'tlo_tx', 'log_sta_tlo', 
            'log_no_abrev', 'bai_no', 'bai_no_abrev', 'mun_nu', 'label_cep']
#, 'cep', 'int_cep'

#features_num = ['log_nu','loc_nu','bai_nu_ini','bai_nu_fim','mun_nu','label_cep']
features_num = ['log_nu','loc_nu','bai_nu_ini','bai_nu_fim','mun_nu']
features_txt = ['ufe_sg', 'log_no', 'log_complemento', 'tlo_tx', 'log_sta_tlo', 
                'log_no_abrev', 'bai_no', 'bai_no_abrev']

#label = ['ufe_sg','label_cep'] 
estratifica = ['ufe_sg','label_cep'] 

In [6]:
X_train, df_end, y_train, y_test = train_test_split(df[features_all], df[estratifica], test_size=0.33, random_state=42)

df_cep = pd.DataFrame()
df_cep['cep'] = df_end['cep']
df = df_end

del X_train, y_test

In [7]:
df_cep.shape

(36921, 1)

In [8]:
def load_stoplist():
    stoplist = open("stopwords.txt", "r")
    stoplist = stoplist.read()
    stoplist = set(stoplist.splitlines())
    return stoplist

stopwords = load_stoplist()

In [9]:
# https://github.com/perinm/PI-2020.1/blob/master/Proj%20Final/Data%20Processing.ipynb
def preprocessing(df):
    return df[features_txt].apply(lambda x: " ".join(x.astype(str)), axis=1)

In [10]:
# https://github.com/perinm/PI-2020.1/blob/master/Proj%20Final/Data%20Processing.ipynb
def parallelize_dataframe(df, func, n_cores=8):
    """
    Function that parallelizes any function applied to a dataframe
    Input:
    df      - Dataframe
    func    - Function to be aplied to portions of Dataframe
    n_cores - Number of CPU cores to be used on the application of func
    
    """
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [11]:
# https://github.com/perinm/PI-2020.1/blob/master/Proj%20Final/Data%20Processing.ipynb
df_end['end_txt'] = parallelize_dataframe(df_end[features_txt], preprocessing)
df_end['end_txt'] = [re.sub(r'nan', ' ', x) for x in df_end['end_txt'].str.lower()]
df_end.shape

(36921, 16)

In [12]:
df_end['cep'] = df_cep.cep
df_end.shape

# Depois de usado o DataFrame df_cep será apagado para liberar de RAM
del df_cep

(36921, 16)

In [13]:
df_end['label'] = [int(x/1000000) for x in df_end.cep] # verificar qtd de variaveis em label

df_end = df_end.drop(columns=['cep'])
#df_end.label.value_counts()
df_end.shape

(36921, 16)

In [14]:
df_end.head(2)

,log_nu,ufe_sg,loc_nu,bai_nu_ini,bai_nu_fim,log_no,log_complemento,tlo_tx,log_sta_tlo,log_no_abrev,bai_no,bai_no_abrev,mun_nu,label_cep,end_txt,label
24505,1046029,MG,3547,4936,0,Professora Maria Hipólita Lemos,NaN,Rua,S,R Prfa Maria H Lemos,Vila São José,Vl S José,3147907,379,mg professora maria hipólita lemos rua s r prfa maria h lemos vila são josé vl s josé,37
57882,1226561,MG,3128,67419,0,Wilson José Barbosa,NaN,Rua,S,R Wilson J Barbosa,Cidade Nova,C Nova,3126109,355,mg wilson josé barbosa rua s r wilson j barbosa cidade nova c nova,35


In [15]:
'''
from sklearn.preprocessing import LabelEncoder

# inicializa LabelEncoder
lencod = LabelEncoder()
# fit_transform() converte texto em número
bbc_news.label = lencod.fit_transform(bbc_news.label)

bbc_news.label.value_counts()
'''

'\nfrom sklearn.preprocessing import LabelEncoder\n\n# inicializa LabelEncoder\nlencod = LabelEncoder()\n# fit_transform() converte texto em número\nbbc_news.label = lencod.fit_transform(bbc_news.label)\n\nbbc_news.label.value_counts()\n'

In [16]:
'''
stemmer = nltk.stem.RSLPStemmer()

end = []
for i in df_end['end_txt'].head():
    e = []
    for j in i.split(' '):
        e.append(stemmer.stem(j))
        
    end.append(e)
    
df_end
'''

"\nstemmer = nltk.stem.RSLPStemmer()\n\nend = []\nfor i in df_end['end_txt'].head():\n    e = []\n    for j in i.split(' '):\n        e.append(stemmer.stem(j))\n        \n    end.append(e)\n    \ndf_end\n"

### 4. TFIDF

O TFIDF também pode ser facilmente implementado em Python usando o Sklearn.

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

# inicializar TFIDF
vec = TfidfVectorizer(max_features=4000, stop_words=stopwords)
# criar TFIDF
tfidf = vec.fit_transform(df_end.end_txt)
#tfidf_df = vec.fit_transform(df_end.end_txt)

tfidf.shape
#tfidf_df.shape

(36921, 4000)

In [18]:
from scipy.sparse import csr_matrix #para transformar DataFrame em Sparse DataFrame otimizado

sp_vec = csr_matrix(tfidf)
tfidf_df = pd.DataFrame.sparse.from_spmatrix(sp_vec)

# amostra
#tfidf_df.iloc[:20, 3000:3050]
tfidf_df.shape

(36921, 4000)

In [19]:
tfidf_df.head()

,0,1,2,3,4,5,6,7,8,9,...,3990,3991,3992,3993,3994,3995,3996,3997,3998,3999
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
tfidf_df = pd.concat([df[['log_nu','loc_nu','bai_nu_ini','bai_nu_fim','mun_nu','label']], tfidf_df], axis=1, join='inner')
tfidf_df.shape

(12170, 4006)

In [21]:
del df_end, df

In [22]:
# cria o modelo ML baseado nos parâmetros
def create_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
               
    model = XGBClassifier()
    model = model.fit(X_train, y_train)
    
    return model, X_test, y_test

In [23]:
from sklearn.metrics import accuracy_score

# criar o modelo de classificação baseado no TFIDF
y_pred, X_test, y_test = create_model(tfidf_df, tfidf_df.label)

acuracia = accuracy_score(y_test, y_pred.predict(X_test))
print("Acurácia: %.2f%%" % (acuracia * 100.0))

Acurácia: 100.00%
